<a href="https://colab.research.google.com/github/Jhansi-27/Convolution-Neural-Networks/blob/main/FashionMNIST_kerastuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install keras-tuner

In [34]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [35]:
print(tf.__version__)

2.4.1


In [36]:
#load fashion MNIST daatset from keras

fashion_mnist = keras.datasets.fashion_mnist 

In [37]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [38]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28)
(10000, 28, 28)


In [39]:
# preprocessing
train_images = train_images/255.0
test_images = test_images/255.0

In [13]:
train_images[0].shape

(28, 28)

In [40]:
# CNN needs 4 dimensions (no of images to train, shape of the image,1)
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [41]:
# creating a function called buildmodel

def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                                kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                                activation = 'relu',
                                input_shape=(28,28,1)
                            ),
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                                kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                                activation = 'relu'
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units=hp.Int('dense_1_units',min_value=32,max_value=64,step=16),
                                activation = 'relu'
                            ),
                            keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [42]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [43]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=5,directory='output',project_name='MNIST_fashion')

INFO:tensorflow:Reloading Oracle from existing project output/MNIST_fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/MNIST_fashion/tuner0.json


In [44]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [68]:
optimized_model=tuner_search.get_best_models(num_models=1)[0]

In [69]:
optimized_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        13872     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1327152   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,341,834
Trainable params: 1,341,834
Non-trainable params: 0
_________________________________________________________________


In [70]:
optimized_model.fit(train_images,train_labels,validation_split=0.1,epochs=10)

Epoch 1/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1877 - accuracy: 0.9312 - val_loss: 0.2410 - val_accuracy: 0.9133
Epoch 2/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1356 - accuracy: 0.9523 - val_loss: 0.2504 - val_accuracy: 0.9162
Epoch 3/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0967 - accuracy: 0.9639 - val_loss: 0.2771 - val_accuracy: 0.9137
Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0653 - accuracy: 0.9765 - val_loss: 0.3141 - val_accuracy: 0.9155
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0480 - accuracy: 0.9829 - val_loss: 0.3584 - val_accuracy: 0.9122
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0313 - accuracy: 0.9891 - val_loss: 0.3873 - val_accuracy: 0.9113
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0248 - accuracy: 0.9915 - val_loss: 0.4223 - val_accuracy:

In [71]:
predictions = optimized_model.predict(test_images)
predictions = np.argmax(predictions,axis=1)
actual_labels = test_labels
predictions[:10]

array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7])

In [72]:
test_labels[:10]

array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7], dtype=uint8)

In [73]:
print("Number of images in test_data: ", len(actual_labels))
count = 0
for i,j in zip(predictions,actual_labels):
  if i != j:
    count+=1
print(f"Out of 10,000 images CNN predicted {count} images incorrectly")

Number of images in test_data:  10000
Out of 10,000 images CNN predicted 879 images incorrectly


In [75]:
879/10000

0.0879